# Mermaid charting 
First we need to load support in the extension lab add on. 

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*

Installed package Microsoft.DotNet.Interactive.ExtensionLab version 1.0.0-beta.21215.5

Loading extensions from `Microsoft.DotNet.Interactive.ExtensionLab.dll`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Explain things visually using the Mermaid language . 
 This extension adds a new kernel that can render Mermaid markdown. This code will render a sequence diagram: 
 
 
#!mermaid
sequenceDiagram
 participant Alice
 participant Bob
 Alice->>John: Hello John, how are you?
 loop Healthcheck
 John->>John: Fight against hypochondria
 end
 Note right of John: Rational thoughts prevail!
 John-->>Alice: Great!
 John->>Bob: How about you?
 Bob-->>John: Jolly good!
 
 
 This extension also adds gestures to render a class diagram from any type. Use the ExploreWithUmlClassDiagram().Display(); extension method on System.Type to render its class diagram. 

 
 
typeof(List ).ExploreWithUmlClassDiagram().Display();

Explore data visually using the nteract Data Explorer . 
 This extension adds the ability to sort, filter, and visualize data using the nteract Data Explorer . Use the ExploreWithNteract() extension method with variables of type JsonElement , IEnumerable or IDataView to render the data explorer. 
 
 
 using Microsoft.Data.Analysis;
 using System.Collections.Generic;
 using Microsoft.ML;

 var dataFrame = DataFrame.LoadCsv("./Data.csv");

 dataFrame.ExploreWithNteract().Display();

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay a notebook session via automation.

Explore data visually using the SandDance Explorer . 
 This extension adds the ability to sort, filter, and visualize data using the SandDance Explorer . Use the ExploreWithSandDance() extension method with variables of type JsonElement , IEnumerable or IDataView to render the data explorer. 
 
 
 using Microsoft.Data.Analysis;
 using System.Collections.Generic;
 using Microsoft.ML;

 var dataFrame = DataFrame.LoadCsv("./Data.csv");

 dataFrame.ExploreWithSandDance().Display();

Query SQLite databases. 
 This extension adds support for connecting to SQLite databases using the #!connect sqlite magic command. For more information, run a cell using the #!sql magic command.

## Magic command
Now we have a magic command for mermaid

In [1]:
#!mermaid
sequenceDiagram
    participant Alice
    participant Bob
    Alice->>John: Hello John, how are you?
    loop Healthcheck
        John->>John: Fight against hypochondria
    end
    Note right of John: Rational thoughts 

## PowerShell 
Define a PowerShell function `Out Mermaid` (with an alias of `Mermaid` so we can call it in more dsl-y style);   
it accepts a mermaid chart definiton as a parameter (example with the definition) or from the   
pipeline (see subsequent example) and  this outputs the minimum correct HTML / Javascript but   
**depends on the kernel extension being loaded** 

Test with examples from [the mermaid home page](https://mermaid-js.github.io/mermaid/#/) -    



In [1]:
function Out-Mermaid {
    [alias('Mermaid')]
    param (
        [parameter(ValueFromPipeline=$true,Mandatory=$true,Position=0)]
        $Text
    )
    begin {
        $mermaid = ""
        $html    =  @"
        <script type="text/javascript">
        configureRequireFromExtension('Mermaid','1.0.0')(['Mermaid/mermaidapi'], (mermaid) => {
            let renderTarget = document.getElementById('~~guid~~');
            mermaid.render('mermaid_~~guid~~',   ``~~Mermaid~~ ``,  g => { renderTarget.innerHTML = g }); 
        }, (error) => { console.log(error);  });
        </script><div  style="background-color:white"; id="~~guid~~"></div>'     
"@  }
    process {
        $Mermaid +=  ("`r`n" + $Text -replace '^[\r\n]+','' -replace '[\r\n]+$','') 
    }
    end     {
        $GUID    =  ([guid]::NewGuid().ToString() -replace '\W','') 
        [Microsoft.DotNet.Interactive.Kernel]::HTML(($html -replace  '~~Mermaid~~',$mermaid -replace '~~guid~~',$GUID )) | Out-Display -MimeType "text/html"
    }
}

Mermaid @'
sequenceDiagram
    participant Alice
    participant Bob
    Alice->>John: Hello John, how are you?
    loop Healthcheck
        John->>John: Fight against hypochondria
    end
    Note right of John: Rational thoughts
'@

'

In [1]:
@'
graph TD;
    A-->B;
    A-->C;
    B-->D;
    C-->D;
'@ | Out-Mermaid

'

In [1]:
Out-Mermaid @'
gantt
dateFormat  YYYY-MM-DD
title Adding GANTT diagram to mermaid
excludes weekdays 2014-01-10

section A section
Completed task            :done,    des1, 2014-01-06,2014-01-08
Active task               :active,  des2, 2014-01-09, 3d
Future task               :         des3, after des2, 5d
Future task2               :         des4, after des3, 5d
'@

'

In [1]:
Mermaid @'
classDiagram
Class01 <|-- AveryLongClass : Cool
Class03 *-- Class04
Class05 o-- Class06
Class07 .. Class08
Class09 --> C2 : Where am i?
Class09 --* C3
Class09 --|> Class07
Class07 : equals()
Class07 : Object[] elementData
Class01 : size()
Class01 : int chimp
Class01 : int gorilla
Class08 <--> C2: Cool label
'@

'

In [1]:
Mermaid @'
erDiagram
    CUSTOMER ||--o{ ORDER : places
    ORDER ||--|{ LINE-ITEM : contains
    CUSTOMER }|..|{ DELIVERY-ADDRESS : uses
'@


'

In [1]:
Mermaid @' 
journey
    title My working day
    section Go to work
      Make tea: 5: Me
      Go upstairs: 3: Me
      Do work: 1: Me, Cat
    section Go home
      Go downstairs: 5: Me
      Sit down: 5: Me
'@

'